# Prolongation of Infinitesimal Generators
***
In this ipython jupyter notebook, we will provide a step by step guide to prolonging infinitesimal generators of a group action.  This will be handled in the python programming language, and we will build different python functions along the way to help break the job into several steps in order to segment the work into understandable segements.

## Introduction ##
***
Although it is outside the scope of this notebook to provide a full background on the prolongation process, it is proably prudent that we review the process of prolonging functions and vector fields.  Throughout this discussion, it will be assumed that the reader has read through sections 1.1-2.3 of Olver.

## Jets ##
***
Next we will develop a few python functions that will be useful for working in $n$-jet space.  For this discussion we will let $X\simeq \mathbb{R}^p$ be the space of independent variables and $U\simeq\mathbb{R}^q$ be the space of dependent variables under consideration, and let $f:X\to U$ be a smooth function. If $f$ is a smooth real valued function, then it has 

$$p_k\equiv \binom{p+k-1}{k}$$

different partial derivatives of order $k$.  If we employ multi-index notation, we can represent the evaluations of these partial derivatives at a given $x\in X$ by the following:

$$ \partial_J f(x)=\dfrac{\partial^k f(x)}{\partial x^{j_1}\cdots\partial x^{j_k}}$$

If we consider the general case when $U\simeq \mathbb{R}^q$ where $q\geq1$, then we recall that $f(x)$ can be given in component functions by $f(x)=(f^1(x),\dots,f^q(x))$, where $f^i:X\to \mathbb{R}$.

* *Note - In order to keep indexing easier, we will denote indices of independent variables with latin characters and we will use Greek characters for indices of dependent variables.*

In the general case, we will denote the evaluation of a particular partial derivative of a component function by 

$$u_J^\alpha = \partial_J f^\alpha(x)$$

Now we denote the space of *all $k$-th order derivatives* of $f:X\to U$ by $U_k$, noting that $U_k\simeq \mathbb{R}^{q\cdot p_k}$.  Finally, we will join these spaces to create the so-called **jet space of $X\times U$** given by:

$$U^{(n)} = U\times U_1\times\cdots \times U_n$$

Given a smooth function $u=f(x)$ with $f:X\to U$, we define the **$n$-th prolongation** of $f$ to be the function $u^{(n)}=\texttt{pr}^{(n)} f(x)$ whose values on each $U_i$ is given by $u_J^\alpha$ as defined above.

* Example - Suppose that $f:\mathbb{R}^2\to \mathbb{R}$, then $$u^{(2)} = \texttt{pr}^{(2)}f(x,y) = (u; u_x, u_y;u_{xx},u_{xy},u_{yy})$$

### Prolonging Functions in Python ###
***
In order to implement the ideas of prolongation in python, we will need to import a two modules and a specific function.  The modules that we will need are

* Numpy - Handles various numerical operations
* Sympy - Handles symbolic mathematics
* combinations_with_replacement - Handle some of the combinatorics later

To import these, we have the following lines of simple code:

In [1]:
import numpy as np
from sympy import *
from itertools import combinations_with_replacement

Now that we have a few of the necessary tools imported, we will begin by building python functions that will take in the space of independent and dependent variables i.e. $X\times U$ and produce the symbolic jet space of $X\times U$, i.e. $U^{(n)}$.

The first such python function that we will build is called $\texttt{jet}$.  This will serve as a building block for producing all of the $k$-th order derivatives of $f$.  This function will take in a list of independent and dependent variables, and return a list which represents all possible partial derivatives of the dependent variables.  The following snippet of code shows how to build this python function.

In [2]:
def jet(X,U):
    J = []
    for i in range(len(U)):
        for j in range(len(X)):
            J.append(U[i]+X[j])
    return J

To see this function in action, lets go through an example.  Suppose that $X\simeq\mathbb{R}^2$ and $U\simeq \mathbb{R}^2$, where $x,y$ are labels for the independent variables and $u,v$ are for the dependent variables.  We apply the python function 'jet' to these spaces with the following command

In [3]:
jet(['x','y'],['u','v'])

['ux', 'uy', 'vx', 'vy']

Next we will build a python function that produces all possible $n$-th order derivatives of a function $f$.  We will call this function $\texttt{njet}$, and it will take in a list of independent and dependent variables as before along with a number $n$ representing the order desired.  The following snippet of code shows how to build this python function

In [4]:
def njet(X,U,n):
    J = U
    for i in range(n):
        J = jet(X,J)
    for i in range(len(J)):
        k = i+1
        for j in range(i+1,len(J)-1):
            if sorted(J[i]) == sorted(J[k]):
                J.remove(J[k])
                k = k-1
            k += 1
        if i >= len(J)-2:
            break            
    return J

Using the same set up as in the last example, let's produce all second order derivatives of a function $u=f(x)$ where $f:X\to U$

In [5]:
njet(['x','y'],['u','v'],2)

['uxx', 'uxy', 'uyy', 'vxx', 'vxy', 'vyy']

This is indeed the expected result.

Now lets extend this idea to finally produce a list of symbols that will represent coordinates on $X\times U^{(n)}$.  To do this, we will build a python function called $\texttt{fulljet}$ that takes in the same data as $\texttt{njet}$, but produces a list of symbols for $X\times U^{(n)}$.

In [6]:
def fulljet(X,U,n):
    J = X+U
    for i in range(n):
        J = J+njet(X,U,i+1)
    return J

Using the same example as above, we will produce the coordinates on $X\times U^{(2)}$.

In [7]:
fulljet(['x','y'],['u','v'],2)

['x',
 'y',
 'u',
 'v',
 'ux',
 'uy',
 'vx',
 'vy',
 'uxx',
 'uxy',
 'uyy',
 'vxx',
 'vxy',
 'vyy']

This will conclude our work for this section.  To recap, we have reviewed the jet space of $X\times U$ and constructed a function $\texttt{fulljet}$ that will provide symbolic coordinates for $X\times U^{(n)}$.  In the next section, we will review the prolongation formula for vector fields and build a function to carry out the process symbolically

## Prolongation Formula for Vector Fields ##
***
In this section, we will not delve into the derivation of the prolongation formula for vector fields.  Instead, we will present the tools necessary for the calculation and give the explicit formula along with the python code to generate the $n$-th prolongation of a given vector field.  We begin with a definition of the total derivative of a smooth function

### Total Derivatives ###
***

**Result** - Suppose $P(x,u^{(n)})$ is a smooth funciton.  The **total derivative** of $P$ with respect to $x^i$ is given by

$$D_i=\dfrac{\partial P}{\partial x^i}+\sum_{\alpha=1}^q u_{J,i}^\alpha \dfrac{\partial P}{\partial u_J^\alpha}$$

where, given $J=(j_1,\dots,j_k)$,

$$u_{J,i}^\alpha = \dfrac{\partial u_J^\alpha}{\partial x^i}=\dfrac{\partial^{k+1}u^\alpha}{\partial x^i\partial x^{j_1}\cdots\partial x^{j_k}}$$

**Example** - Suppose that $X=\mathbb{R}$ and $U=\mathbb{R}^2$ with coordinates on $U$ being given by $(u,v)=(u^1,u^2)$, then we have

$$D_xP = \dfrac{\partial P}{\partial x}+ u_x\dfrac{\partial P}{\partial u}+v_x\dfrac{\partial P}{\partial v}+u_{xx}\dfrac{\partial P}{\partial u_x}+v_{xx}\dfrac{\partial P}{\partial v_x}+\cdots$$

With this in mind, we can define the $J$-th total derivative operator as follows:

**Definition** - Given a multi-index $J=(j_1,\dots,j_k)$ we define $D_J$ to be given by 

$$D_J = D_{j_1}D_{j_2}\cdots D_{j_k}.$$

Before we go any further, lets see how to keep track of these operations symbolically in python.  We will begin with a python function to generate a term corresponding to $u_{J,i}^\alpha$.  This is handled by the following functions:

In [8]:
def uJalpha(u_div,x_var,X,U):
    J = njet(X,U,len(str(u_div)))
    for i in range(len(J)):
        if sorted(str(u_div)+str(x_var)) == sorted(J[i]):
            return var(J[i])
            break
            
def divUJi(X,U,u,J,i):
    f = u
    for j in range(len(J)):
        f = uJalpha(f,X[J[j]],X,U)
    f = uJalpha(f,X[i],X,U)
    return f

The first function defined above takes in a value $\texttt{u_div}$ corresponding to some $u^\alpha_J$ and takes a derivative with respect to the variable $\texttt{x_var}$.  In addition, this function returns an ordered partial derivative, i.e. if $X=[\texttt{ x , y }]$ then it is possible to obtain $u_{xy}$ but not $u_{yx}$.  The ordering is determined by the order of variables as given in the list $X$.  To get a sense of how this works, take a look at the following simple command:

In [12]:
uJalpha('uy','x',['x','y'],['u'])

uxy

The second function uses the first in order to obtain an ordered expression for the $u_{J,i}^\alpha$ term given in formula for $D_i$.  Note that the input $J$ is given by a list of integers and the input $i$ is given by an integer as well.  These integers correspond to variables in the list $X$ at a given position.  *Note - in python list positions start with the number 0 not 1*. To see this function in action, take a look at the following command

In [13]:
divUJi(['x','y'],['u'],'u',[0,1],1)

uxyy

With these python functions defined, we can move on to building funcitons that return the total derivatives.  Our first task is to make a function called $\texttt{divUJi}$ that takes in the independent and dependent variables, a $u^\alpha$, a multi-index $J$, and an integer $i$, while outputing a value for $u_{J,i}^\alpha$.  This is done with the follow code:

In [14]:
def divUJi(X,U,P,J,i):
    for j in range(len(J)):
        P = uJalpha(P,X[J[j]],X,U)
    P = uJalpha(P,X[i],X,U)
    return u

This function implements the formula for $D_i$ as given in the **Result** above.  To see an example of this, we will compute $D_xP(x,u^{(2)})$ from the example above, where $U\simeq \mathbb{R}^2$ with coordinates $(u,v)=(u^1,u^2)$.

In [19]:
divUJi(['x','y'],['u'],'u',[1,0],1)

uxyy

With these python functions defined, we can move on to building funcitons that return the total derivatives.  Our first task is to make a function called $\texttt{divUJi}$ that takes in the independent and dependent variables, a $u^\alpha$, a multi-index $J$, and an integer $i$, while outputing a value for $u_{J,i}^\alpha$.  This is done with the follow code:

In [20]:
def totDiv(X,U,P,i,n):
    expr = diff(P,X[i])
    J = filter(lambda v: v not in X, fulljet(X,U,n))
    for j in range(len(J)):
        expr = expr + diff(P,J[j])*uJalpha(J[j],X[i],X,U)
    return expr

This function implements the formula for $D_i$ as given in the **Result** above.  To see an example of this, we will compute $D_xP$ for $P=xuu_{xy}$ with $X\simeq \mathbb{R}$ and $U\simeq \mathbb{R}$.

In [27]:
totDiv(['x','y'],['u'],'x*u*uxy',0,2)

u*uxxy*x + u*uxy + ux*uxy*x

We are now at the point that we can define $D_J$ by sucessively applying our previously defined python function.  This is handled easily by the following snippet of code:

In [30]:
def TotDiv(X,U,P,J):
    expr = totDiv(X,U,P,J[0],len(J)+1)
    for i in range(1,len(J)):
        expr = totDiv(X,U,expr,J[i],len(J)+1)
    return expr

In the following example, we comput $D_{xy}P$ for $P=xuu_{xy}$.

In [34]:
TotDiv(['x','y'],['u'],'x*u*uxy',[0,1])

u*uxxyy*x + uxy**2*x + uxyy*(u + ux*x) + uy*(uxxy*x + uxy)

Having developed much of the tools necessary to prolong vector fields, we will now segue to the prolongation formula for vector fields

## General Prolongation Formula ##
***
We begin this section with the formula for prolonging a vector field of the form:

$$v = \sum_{i=1}^p\xi^i(x,u)\dfrac{\partial}{\partial x^i}+\sum_{\alpha=1}^q \phi_\alpha(x,u)\dfrac{\partial}{\partial u^\alpha} $$

**Result** The $n$-th prolongation of the vector field above is given by the following formula

$$\texttt{pr}^{(n)}v=v+\sum_{\alpha=1}^q\sum_{|J|\leq n} \phi_\alpha^J(x,u^{(n)})\dfrac{\partial}{\partial u_J^\alpha}$$

where the coefficient functions $\phi_\alpha^J$ are given by:

$$\phi_\alpha^J(x,u^{(n)}) = D_J\left(\phi_\alpha-\sum_{i=1}^p\xi^i u_i^\alpha\right)+\sum_{i=1}^p\xi^iU_{J,i}^\alpha $$

and where $u_i^\alpha=\partial u^\alpha/\partial x^i$, and $u^\alpha_{J,i}$ is defined as above.

We will implement this calculation in two parts, we first define a python function called $\texttt{phiAlpha}$ that takes in a list of independent and dependent variables, a vector field (*in list form*), and a multi-index $J$.  This is accomplished as follows

In [36]:
def phiAlpha(X,U,v,J):
    phi = v[len(X):]
    xi = v[:len(X)]
    a = 0
    q = len(U)-1
    for i in range(len(X)):
        a = a+xi[i]*uJalpha(U[q],X[i],X,U)
    b = 0
    for i in range(len(X)):
        b = b+xi[i]*divUJi(X,U,U[q],J,i)
    c = TotDiv(X,U,phi[q]-a,len(J),J)+b
    return c

With this completed, we finally can define a python function called $\texttt{prolong}$ that takes in a list of independent and dependent variables, a vector field $v$ (in *list form*), and an integer $n$ and the function returns the $n$-th prolongation of the vector field $v$.  This is accomplished as follows

In [37]:
def Prolong(X,U,v,n):
    a = v
    J = diffOrd(len(X),n)
    for i in range(n):
        for j in range(len(X)):
            a.append(phiAlpha(X,U,v,J[i][j]))
    return a

In order to take full advantage of this, we will implement the process of finding the prolongation of a vector field in a more user friendly way with the following snippet of code

In [38]:
if __name__ == "__main__":
    
    #initialize symbols for sympy
    p = input("Input the number of independent variables:")
    q = input("Input the number of dependent variables:")
    X = []
    U = []
    for i in range(p):
        X.append(raw_input("What label would you like to assign to independent variable "+str(i+1)+"?"))
    for j in range(q):
        U.append(raw_input("What label would you like to assign to dependent variable "+str(j+1)+"?"))
    var(fulljet(X,U,3))
    P = filter(lambda v: v not in X, fulljet(X,U,3))

Input the number of independent variables:2
Input the number of dependent variables:2
What label would you like to assign to independent variable 1?x
What label would you like to assign to independent variable 2?y
What label would you like to assign to dependent variable 1?u
What label would you like to assign to dependent variable 2?v


If we put this all together in one easy to use program, we have the following:

In [ ]:
import numpy as np
from sympy import *
from itertools import combinations_with_replacement

def jet(X,U):
    J = []
    for i in range(len(U)):
        for j in range(len(X)):
            J.append(U[i]+X[j])
    return J
                
def njet(X,U,n):
    J = U
    for i in range(n):
        J = jet(X,J)
    for i in range(len(J)):
        k = i+1
        for j in range(i+1,len(J)-1):
            if sorted(J[i]) == sorted(J[k]):
                J.remove(J[k])
                k = k-1
            k += 1
        if i >= len(J)-2:
            break            
    return J

def fulljet(X,U,n):
    J = X+U
    for i in range(n):
        J = J+njet(X,U,i+1)
    return J

def uJalpha(u,x,X,U):
    J = njet(X,U,len(str(u)))
    for i in range(len(J)):
        if sorted(str(u)+str(x)) == sorted(J[i]):
            return var(J[i])
            break
            
def divUJi(X,U,u,J,i):
    f = u
    for j in range(len(J)):
        f = uJalpha(f,X[J[j]],X,U)
    f = uJalpha(f,X[i],X,U)
    return f


def totDiv(X,U,P,i,n):
    expr = diff(P,X[i])
    J = filter(lambda v: v not in X, fulljet(X,U,n))
    for j in range(len(J)):
        expr = expr + diff(P,J[j])*uJalpha(J[j],X[i],X,U)
    return expr

def TotDiv(X,U,P,n,J):
    expr = totDiv(X,U,P,J[0],n)
    for i in range(1,len(J)):
        expr = totDiv(X,U,expr,J[i],n)
    return expr

def phiAlpha(X,U,v,J):
    phi = v[len(X):]
    xi = v[:len(X)]
    a = 0
    q = len(U)-1
    for i in range(len(X)):
        a = a+xi[i]*uJalpha(U[q],X[i],X,U)
    b = 0
    for i in range(len(X)):
        b = b+xi[i]*divUJi(X,U,U[q],J,i)
    c = TotDiv(X,U,phi[q]-a,len(J),J)+b
    return c

def diffOrd(p,n):
    b = []
    for i in range(1,n+1):
        b.append(list(combinations_with_replacement(range(p),i)))
    return b

def Prolong(X,U,v,n):
    a = v
    J = diffOrd(len(X),n)
    for i in range(n):
        for j in range(len(X)):
            a.append(phiAlpha(X,U,v,J[i][j]))
    return a
        

if __name__ == "__main__":
    
    #initialize symbols for sympy
    p = input("Input the number of independent variables:")
    q = input("Input the number of dependent variables:")
    X = []
    U = []
    for i in range(p):
        X.append(raw_input("What label would you like to assign to independent variable "+str(i+1)+"?"))
    for j in range(q):
        U.append(raw_input("What label would you like to assign to dependent variable "+str(j+1)+"?"))
    var(fulljet(X,U,3))
    P = filter(lambda v: v not in X, fulljet(X,U,3))


## References ##
***
1. Peter Olver, *Applications of Lie Groups to Differential Equations*, 2nd Edition, Springer-Verlag 1993